In [3]:
from libdatasets import *
import librun
import libstop
from libadversarial import uncertainty_stop
from libstop import rank_stop_conds
from dotenv import load_dotenv; load_dotenv();

matrix = {
    # Dataset fetchers should cache if possible
    # Lambda wrapper required for function to be pickleable (sent to other threads via joblib)
    "datasets": [
        ("rcv1", wrap(rcv1, None)),
        ("webkb", wrap(webkb, None)),
        ("spamassassin", wrap(spamassassin, None)),
        ("avila", wrap(avila, None)),
        ("smartphone", wrap(smartphone, None)),
        ("swarm", wrap(swarm, None)),
        ("sensorless", wrap(sensorless, None)),
        ("splice", wrap(splice, None)),
        ("anuran", wrap(anuran, None)),
    ],
    "dataset_mutators": {
        "none": (lambda *x, **kwargs: x),
    },
    "methods": [
        ("uncertainty", partial(uncertainty_stop, n_instances=10)),
    ],
    "models": [
        "svm-linear",
        #"decision-tree",
        #"random-forest",
    ],
    "meta": {
        "dataset_size": 1000,
        "labelled_size": 10,
        "test_size": 0.5,
        "n_runs": 10,
        "ret_classifiers": True,
        "ensure_y": True,
        "stop_info": True,
        "aggregate": False,
        "stop_function": ("len1000", lambda learner: learner.y_training.shape[0] >= 1000),
        "pool_subsample": 1000
    }
}

results = librun.run(matrix, force_cache=True, fragment_run_start=0, fragment_run_end=9)
results_plots = [result[0] for result in results]
classifiers = [result[1] for result in results]
classifiers = [clf for clf in classifiers]

for plots, clfs in zip(results_plots, classifiers):
    for i, clfs_ in enumerate(clfs):
        if len(clfs_) != 100:
            raise Exception(f"{plots[0].serialize()}_{i}.zip")

from libstop import *
params = {
    "kappa": {"k": 2}
}
conditions = {
    **{f"{f.__name__}": partial(f, **params.get(f.__name__, {})) for f in [
        uncertainty_min, 
        SC_entropy_mcs, 
        SC_oracle_acc_mcs, 
        SC_mes,
        EVM, 
        #ZPS_ee_grad, 
        stabilizing_predictions
    ]},
    "ZPS2": partial(ZPS, order=2),
    "SSNCut": SSNCut
}

['C:\\Users\\Zac\\AppData\\Roaming\\Python\\Python38\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\Zac\\AppData\\Roaming\\jupyter\\runtime\\kernel-1218c780-56ad-4949-84ea-54e845f172a6.json']


In [7]:
def eval_stopping_conditions(results_plots, classifiers, conditions=None):
    if conditions is None:
        params = {
            "kappa": {"k": 2}
        }
        conditions = {
            **{f"{f.__name__}": partial(f, **params.get(f.__name__, {})) for f in [
                uncertainty_min, 
                SC_entropy_mcs, 
                SC_oracle_acc_mcs, 
                SC_mes,
                EVM, 
                #ZPS_ee_grad, 
                stabilizing_predictions
            ]},
            "ZPS2": partial(ZPS, order=2),
            #"SSNCut": SSNCut
        }

    stop_results = {}
    
    def eval_cond(name, conf, cond, j, **kwargs):
        #print(f"exec_cond Execing {name} on {conf.dataset_name}")
        if name in stop_results[conf.dataset_name] and len(stop_results[conf.dataset_name][name]) > j:
            return stop_results[conf.dataset_name][name][j][0]
        try:
            return cond(**kwargs)
        except FailedToTerminate:
            print(f'{name} failed to terminate on {conf.dataset_name} run {j}')
            return None
        except Exception as e:
            print(f'WARNING {name} failed on {conf.dataset_name} run {j} with exception: {e}')
            return None
    
    for (clfs, (conf, metrics)) in zip(classifiers, results_plots):
        stop_results[conf.dataset_name] = __read_stopping(conf.serialize())
                
        # todo: split this into chunks for memory usage reasons
        stop_results[conf.dataset_name]["SSNCut"] = [
            (50.0, 0.5172386358268255, 0.6612975251013473, 0.8555689022254009),
            (90.0, 0.8225366540114916, 0.8026749750499692, 0.9025002286378632),
            (60.0, 0.6346483278510816, 0.7185090540600079, 0.8974647979420125),
            (100.0, 0.7948096378233099, 0.7505584017167644, 0.8930116627475733),
            (80.0, 0.640583082840428, 0.3913468177914007, 0.8919311563211882),
            (100.0, 0.78653031896511, 0.7305665824612053, 0.8936941456655085),
            (80.0, 0.8001377400194427, 0.7465555184220649, 0.9189423977826924),
            (80.0, 0.6997466478703752, 0.7487433422103861, 0.8876128818883928),
            (90.0, 0.8265147051145306, 0.7997566442155649, 0.9086930964786852),
            (70.0, 0.795090587682462, 0.7743708797827372, 0.8668501813072356)
        ]
        
        for i in range(len(conditions)):
            stop_results[conf.dataset_name][list(conditions.keys())[i]] = [(
                x if list(conditions.keys())[i] != 'SSNCut' else x+10, 
                metrics[j]['accuracy_score'][metrics[j].x==x].iloc[0] if x is not None else None, 
                metrics[j]['f1_score'][metrics[j].x==x].iloc[0] if x is not None else None, 
                metrics[j]['roc_auc_score'][metrics[j].x==x].iloc[0] if x is not None else None
            ) for j, x in enumerate(results[:,i])]
        #__write_stopping(conf.serialize(), stop_results[conf.dataset_name])
            
    return (conditions, stop_results)


In [8]:
stop_conditions, stop_results = libstop.eval_stopping_conditions(results_plots, classifiers, conditions=conditions)

c:\users\zac\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\users\zac\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\users\zac\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
c:\users\zac\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.23.2 when using vers

KeyboardInterrupt: 